In [1]:
Rm = 770

In [2]:
#Rm = 1995

In [3]:
import pandas as pd
#from .rambgood import RambergOsgood
#from .hardness import TabularHardnessConverter


class UML:
    _Rm_default = {"STEEL": 1100., "AL": 200., "TI": 300.}
    _sigf_fact = {"STEEL": 1.5, "AL": 1.67, "TI": 1.67}
    _b = {"STEEL": -0.087, "AL": -0.095, "TI": -0.095}
    _c = {"STEEL": -0.58, "AL": -0.69, "TI": -0.69}
    _epsf_fact_a = {"STEEL": 0.0, "AL": 0.35, "TI": 0.35}
    _epsf_fact_b = {"STEEL": 0.59, "AL": 0.0, "TI": 0.0}
    _K_fact = {"STEEL": 1.65, "AL": 1.61, "TI": 1.61}
    _n = {"STEEL": 0.15, "AL": 0.11, "TI": 0.11}

    def __init__(self, material="STEEL", E=210000., Rm=None, Rm_from="Rm"):
        """
        Initialize the Universal Material Law

        Parameters
        ----------
        material : String
            optional, material identifier, one of
                * "STEEL" : non-alloyed or low-alloyed steel
                * "AL" : aluminum alloys
                * "TI" : titane alloys
            default : "STEEL"
        E : float
            optional, YOUNGs modulus for the material (only relevant for `material == "STEEL"` for the parameter estimation)
            default : 210000.0
        Rm : float or None
            optional, the tensile strength of the material
            if `None`, then the following values are used
                * `material == "STEEL"`: 1100 MPa
                * `material == "AL"`: 200 MPa
                * `material == "TI"`: 300 MPa
            default : None
        Rm_from : String
            optional, specify the units in which `Rm` is given, one of
                * "Rm" : strength given in MPa
                * "HV", "HB", "HRA", "HRB", "HRC" : value is estimated from corresponding hardness
            default : "Rm"
        """
        self._material = material
        self._E = E
        self._Rm = self._calc_Rm(Rm, Rm_from)

    def _calc_Rm(self, Rm, Rm_from):
        if Rm is None:
            return self._Rm_default[self._material]
        if Rm_from == "Rm":
            return Rm

        hard = TabularHardnessConverter(cfrom=Rm_from, cto="Rm")
        return hard.convert(Rm)

    def _calc_sigf(self):
        return self._sigf_fact[self._material] * self._Rm

    def _calc_b(self):
        return self._b[self._material]

    def _calc_psi(self):
        if self._material != "STEEL":
            return 0.0
        Rm_norm = self._Rm / self._E
        if Rm_norm <= 3e-3:
            return 1.0
        return 1.375 - 125. * Rm_norm

    def _calc_epsf(self):
        return self._epsf_fact_a[self._material] + self._epsf_fact_b[self._material] * self._calc_psi()

    def _calc_c(self):
        return self._c[self._material]

    def _calc_K(self):
        return self._K_fact[self._material] * self._Rm

    def _calc_n(self):
        return self._n[self._material]

    @property
    def cmb(self):
        """
        Get COFFIN-MANSON-BASQUIN parameters
        """
        return pd.Series({
            "sigf": self._calc_sigf(),
            "epsf": self._calc_epsf(),
            "c": self._calc_c(),
            "b": self._calc_b()
        })

    @property
    def cyclic(self):
        """
        Get cyclic parameters (K' and n')
        """
        return pd.Series({
            "K": self._calc_K(),
            "n": self._calc_n()
        })

    def to_rambgood(self):
        """
        Get Ramberg-Osgood from Universal Material Law
        """
        cyclic = self.cyclic
        return RambergOsgood(E=self._E, K=cyclic["K"], n=cyclic["n"])

In [12]:
cl_uml = UML(Rm=Rm)

In [13]:
CMB_Params = cl_uml.cmb


In [14]:
CMB_Params

sigf    1155.000000
epsf       0.540833
c         -0.580000
b         -0.087000
dtype: float64

In [15]:
eps_pcrit = 0.574

In [16]:
def pw_m(c):
    return -1/c

In [17]:
m_pl = pw_m(CMB_Params['c'])
m_pl

1.7241379310344829

In [18]:
def FIP_critpl_eq(epsf,c,eps_crit):
    zahler = 4*epsf*(2**c)
    return zahler/eps_crit

In [19]:
FIP_crit_pl = FIP_critpl_eq(CMB_Params['epsf'],CMB_Params['c'], eps_pcrit)
FIP_crit_pl

2.5212397882003104

Interpolate hardnessconversionlist 37HRC

In [33]:
Rm_i = ((37-36.6)/(37.7-36.6))*(1190-1155)+1155

In [34]:
Rm_i

1167.7272727272727